In [ ]:
from dataretrieval import nwis
from dataretrieval import wqp

# NWM dataset 
# https://github.com/NOAA-OWP/hydrotools?tab=readme-ov-file


## Data source, availability, and information
From upstream to downsteam
### 1. Skagit River Above Miller Creek Near Rockport, WA - 12189700
source: https://waterdata.usgs.gov/monitoring-location/12189700/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height, feet (from 5/2020)
Latitude 48°28'50.80",   Longitude 121°39'04.70"   NAD83
Skagit County, Washington, Hydrologic Unit 17110005
Datum of gage: 199.93 feet above   NAVD88.


### 2. Lake Shannon at Concrete, WA - 12193000 
source: https://waterdata.usgs.gov/monitoring-location/12193000/#dataTypeId=continuous-62615-0&period=P7D&showMedian=true
Availability: Gage height, feet (from 5/2020)
Latitude 48°32'53",   Longitude 121°44'22"   NAD27
Skagit County, Washington, Hydrologic Unit 17110005
Drainage area: 297 square miles
Datum of gage: 0. feet above   NAVD88.
Note: Location inside the dam

### 3. Baker River at Henry Thompson BR at Concrete, WA - 12193400 
Note: inflow discharge B
source: https://waterdata.usgs.gov/monitoring-location/12193400/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height, discharge
Latitude 48°32'26",   Longitude 121°44'32"   NAD27
Skagit County, Washington, Hydrologic Unit 17110005
Drainage area: 297 square miles
Datum of gage: 174.08 feet above   NAVD88.

### 4. Skagit River Near Concrete, WA - 12194000
Note: inflow discharge C
source: https://waterdata.usgs.gov/monitoring-location/12194000/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height, discharge
Latitude 48°31'28",   Longitude 121°46'11"   NAD27
Skagit County, Washington, Hydrologic Unit 17110007
Drainage area: 2,737 square miles
Datum of gage: 133.96 feet above   NAVD88.

### 5. Finney Creek at South Skagit Hwy NR Concrete, WA - 12194580
source: https://waterdata.usgs.gov/monitoring-location/12194580/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height
Latitude 48°31'17.91",   Longitude 121°50'24.04"   NAD83
Skagit County, Washington, Hydrologic Unit 17110007
Datum of gage: 133 feet above   NAVD88.
k at So
### 6. Skagit River Near Sedro Woolley, WA - 12199000
source: https://waterdata.usgs.gov/monitoring-location/12199000/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height 
Latitude 48°29'03",   Longitude 122°14'31"   NAD27
Skagit County, Washington, Hydrologic Unit 17110007
Drainage area: 3,015 square miles
Datum of gage: 3.81 feet above   NAVD88.

### 7. Skagit River Near Mount Vernon, WA - 12200500
source: https://waterdata.usgs.gov/monitoring-location/12200500/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height, discharge, temperature, turbidity (9/2016 - 10/2017) 
Latitude 48°26'42",   Longitude 122°20'03"   NAD27
Skagit County, Washington, Hydrologic Unit 17110007
Drainage area: 3,093 square miles
Datum of gage: 3.80 feet above   NAVD88.

### 8. Sauk River Near Sauk, WA - 12189500
Note: Can be used as temperature input
source: https://waterdata.usgs.gov/monitoring-location/12189500/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
Availability: Gage height, discharge, temperature, turbidity (9/2011 - 4/2021) 
Latitude 48°25'29",   Longitude 121°34'02"   NAD27
Skagit County, Washington, Hydrologic Unit 17110006
Drainage area: 714 square miles
Datum of gage: 266 feet above   NGVD29.

### 8. Skagit River at Marblemount, WA - 12181000
Note: Can be used as temperature input
source: https://waterdata.usgs.gov/monitoring-location/12181000/#dataTypeId=continuous-00065-85281465&period=P7D&showMedian=false
Availability: Gage height, discharge, temperature
Latitude 48°32'02",   Longitude 121°25'43"   NAD27
Skagit County, Washington, Hydrologic Unit 17110005
Drainage area: 1,381 square miles
Datum of gage: 308.64 feet above   NAVD88.

In [9]:



siteNumbers = ["01491000","01645000"]
siteINFO, md = nwis.get_iv(sites=siteNumbers)

In [10]:
dailyDataAvailable, md = nwis.get_dv(sites=siteNumbers, statCd="00003")

In [11]:
# Choptank River near Greensboro, MD:
siteNumber = "01491000"
parameterCd = "00060"  # Discharge
startDate = "2009-10-01"  
endDate = "2012-09-30" 

discharge, md = nwis.get_dv(sites=siteNumber, parameterCd=parameterCd, start=startDate, end=endDate)

In [12]:
siteNumber = "01491000"
parameterCd = ["00010","00060"]  # Temperature and discharge
statCd = ["00001","00003"]  # Mean and maximum
startDate = "2012-01-01"
endDate = "2012-05-01"

temperatureAndFlow, md = nwis.get_dv(sites=siteNumber, parameterCd=parameterCd,
                                     start=startDate, end=endDate, statCd=statCd)

In [13]:
parameterCd = "00060"  # Discharge
startDate = "2012-05-12" 
endDate = "2012-05-13" 
dischargeUnit, md = nwis.get_iv(sites=siteNumber, parameterCd=parameterCd,
                            start=startDate, end=endDate)

In [14]:
siteNumber = "01491000"
parameterCd = ["00618","71851"]
startDate = "1985-10-01"
endDate = "2012-09-30"
dfLong, md = nwis.get_qwdata(sites=siteNumber, parameterCd=parameterCd,
                             start=startDate, end=endDate)

/opt/conda/envs/ssmc/lib/python3.11/site-packages/dataretrieval/nwis.py:193: UserWarning: WARNING: Starting in March 2024, the NWIS qw data endpoint is retiring and no longer receives updates. For more information, refer to https://waterdata.usgs.gov.nwis/qwdata and https://doi-usgs.github.io/dataRetrieval/articles/Status.html or email CompTools@usgs.gov.
  warnings.warn(


KeyError: 'sample_start_time_datum_cd'

In [ ]:
siteNumber = "434400121275801"
groundWater, md = nwis.get_gwlevels(sites=siteNumber)

In [ ]:
ratings_data, md = nwis.get_ratings(site='01594440', file_type="base")

In [ ]:
siteNumber = '01594440'
surface_data, md = nwis.get_discharge_measurements(sites=siteNumber)

In [ ]:
allegheny, md = nwis.get_water_use(state="PA", counties="003")

national, md = nwis.get_water_use()

In [ ]:
discharge_stats, md = nwis.get_stats(sites='02319394', parameterCd="00060", statReportType='annual', statTypeCd='all')

In [ ]:
# specific_cond, md = wqp.get_results(siteid='WIDNR_WQX-10032762',
#                           characteristicName = 'Specific conductance',
#                           startDateLo='2011-05-01', startDateHi='2011-09-30')

In [ ]:
# dischargeWI, md = nwis.get_dv(stateCd="WI", parameterCd="00060", drainAreaMin="50", statCd="00003")

In [ ]:
# sitesNJ, md = wqp.what_sites(statecode="US:34", characteristicName="Chloride")

In [ ]:
# dataPH, md = wqp.what_sites(statecode="US:55", characteristicName="pH")

In [ ]:
site, md = wqp.what_sites(siteid="USGS-01594440")

In [ ]:
streamType = "Stream"
sites, md = wqp.what_sites(countycode="US:55:025",siteType=streamType)